In [1]:
# %cd ..
!nvidia-smi

Sat Apr 11 06:36:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install gdown
!pip install tensorflow-gpu
!pip install --upgrade keras
 

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 3.9MB 55.0MB/s 
     |████████████████████████████████| 450kB 51.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4ef8256586b573aaeadff4623c29f6ce15d183b2cafc9d34c36853642c070291
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.0
    Uninstal

In [3]:
%cd /content/drive/My\ Drive/person_detection/keras-retinanet
!pip install .
!python setup.py build_ext --inplace
%cd ../../../../sample_data/
!git clone https://github.com/Yannick947/WiderPerson.git
%cd ../../

/content/drive/My Drive/person_detection/keras-retinanet
Processing /content/drive/My Drive/person_detection/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=171937 sha256=04bd55e3d28d018ca0c7d747a65c8cfb3a80efa570cb2ce3c6f2e8d439d36543
  Stored in directory: /root/.cache/pip/wheels/ca/78/63/58275b4a0f024168bf718212448cbba87f45aa1948fdc1a291
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=ebe791c72fb44280d6595ac352e8c0ef7b4101ea99cf0fbb9d2d19ac8f188398
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet
running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.6/keras_retinanet/utils/compute_overlap.cpython-36m-x86_64-linux-gnu.so -> keras_retinanet/utils
/content/sample_data
Cloning into 'Wid

In [4]:
#quickfix error in callbacks of keras, pullrequest is already open in github
#later versions of tensorflow and keras should contain this fix

from google.colab import files
import shutil
%cd /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/
# files.download('callbacks.py')
!rm callbacks.py
shutil.copyfile('/content/drive/My Drive/person_detection/utils/callbacks.py', 
            './callbacks.py')
!ls
%cd ../../../../../../../

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras
activations.py	   estimator			   optimizers.py
api		   initializers.py		   optimizer_v2
applications	   __init__.py			   premade
backend_config.py  keras_parameterized.py	   preprocessing
backend.py	   layers			   __pycache__
callbacks.py	   losses.py			   regularizers.py
callbacks_v1.py    metrics.py			   saving
constraints.py	   mixed_precision		   testing_utils.py
datasets	   models.py			   utils
distribute	   model_subclassing_test_util.py  wrappers
engine		   ops.py
/usr


In [5]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from IPython.core.debugger import set_trace
import shutil
from time import gmtime, strftime
import pdb

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
annot_path = '/content/drive/My Drive/person_detection/WiderPerson/Annotations'
images_path = '/content/drive/My Drive/person_detection/WiderPerson/Images'
annot_csv_path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations_new.csv'
keras_path = '/content/drive/My Drive/person_detection/keras-retinanet'
#Remove classes which shall be left out in the csv
classes_ids = {1:'pedestrian',
               2:'riders',
               3:'partially-visible',
               5:'crowd'}

# Training



In [0]:
PRETRAINED_RES50_VANILLA = './snapshots/resnet50_vanilla.h5'
PRETRAINED_RES152_VANILLA = './snapshots/resnet152_vanilla_oid.h5'
PRETRAINED_MODEL_WI = './snapshots/09-04-2020_101444_mAP61_res50.h5'
logdir = 'tensorboard/{}'.format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
#hyperparams which will get logged for tensorboard use
hyperparams = {'image_min_side':100,
               'image_max_side': 1500,
               'weights':PRETRAINED_RES50_VANILLA, 
               'lr': 1e-5, 
               'batch_size': 4, 
               'step_size': 1000, 
               'epochs': 40,
               'images_filtered': '>600'
               }


In [0]:
%cd /content/drive/My Drive/person_detection/keras-retinanet
!python keras_retinanet/bin/train.py  --no-resize\
                                      --backbone 'resnet50'\
                                      --weights {hyperparams['weights']} \
                                      --lr {hyperparams['lr']}\
                                      --batch-size {hyperparams['batch_size']} \
                                      --steps {hyperparams['step_size']} \
                                      --epochs {hyperparams['epochs']} \
                                      --weighted-average \
                                      --tensorboard-dir {logdir}\
                                      --filtered-above 600\
                                      --random-transform \
                                      --compute-val-loss\
                                      --num-classes 1\
                                      --config config.ini\
                                      csv annot_train_600_2500_classes_filtered.csv classes_filtered.csv \
                                      --val-annotations annot_test_600_2500_classes_filtered.csv \
                                      # --config config.ini\
                                      #'--imagenet-weights' \
%cd ../../../../../

/content/drive/My Drive/person_detection/keras-retinanet
Using TensorFlow backend.
2020-04-11 06:46:11.565499: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-04-11 06:46:11.565587: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-04-11 06:46:11.565602: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
annot_test_600_2500_classes_filtered.csv
Creating model, this may take a second...
2020-04-11 06:46:13.377926: W tensorflow/co

### Anchor Optimization

In [0]:
%cd /content/drive/My Drive/person_detection
!git clone https://github.com/martinzlocha/anchor-optimization.git
%cd anchor-optimization
!pip install .
!python setup.py build_ext --inplace

In [0]:
!python ./../anchor-optimization/optimize_anchors.py annot_train.csv


## Functions


In [0]:
# Generate the classes csv file
with open(annotations, newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for filename in os.listdir(images_path)[0:10]:
    if str(filename + '.txt') in annot:
      f = open(annot_path + '/' +  filename + '.txt', 'r')
      
      for index, line in enumerate(f): 
        if index == 0: 
          if line.strip() == '0':
            print('Not any  object in the image!')
          continue
          
        else: 
          split_line = line[:line.find('/')].split(' ')
          first_char = split_line.pop(0)
          split_line.insert(len(split_line), first_char)
          split_line.insert(0, images_path + '/' + filename)
          #convert from index to class label 
          try:
            split_line[-1] = classes_ids[int(split_line[-1])]
          except: 
            continue
          split_line[0] = split_line[0]

          csv_writer.writerow(split_line)
      
      f.close()

#create indexing csv file
with open('classes.csv', newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for index, key in enumerate(classes_ids.keys()):
    csv_writer.writerow([str(classes_ids[key]), index])


#Check for wrongly annotaded bounding boxes
path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations.csv'
def check_bb(path):
  colnames = ['filename', 'x1', 'y1', 'x2', 'y2', 'class_label']
  df = pd.read_csv(path, names=colnames)
  df_new = df.loc[(df.x1 < df.x2) & (df.y1 < df.y2), : ]
  print ('Reduces shape from ', df.shape, 'to ', df_new.shape)
  return df_new





In [0]:
def replace_annoation_folder(new_folder, annot_path, old_img_path):
  '''Start this function from folder root, otherwise wont work properly'''

  df_annot = pd.read_csv(annot_path + '/annotations_pathkeras.csv',
                         names=['name', 'x1', 'y1', 'x2', 'y2', 'label'])
  df_annot = df_annot.reset_index().drop(0).drop(columns='index')
  df_annot['name'] = df_annot['name'].str.replace(old_img_path, new_folder)
  df_annot = df_annot.dropna()
  df_annot.to_csv(annot_path + '/annotations_new.csv',
                  index=None, header=None)
  return


old_folder = '/content/drive/My Drive/PersonDetection/WiderPerson/Images'
new_folder = '/content/drive/My Drive/person_detection/WiderPerson/Images'
annot_path_csv = '/content/drive/My Drive/person_detection/keras-retinanet'
replace_annoation_folder(new_folder, annot_path_csv, old_folder)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [0]:
#Splitting into train and test data

train_df, test_df = train_test_split(
  pd.read_csv(annot_csv_path, header=None), 
  test_size=0.15, 
  random_state=RANDOM_SEED
)

train_df.to_csv(keras_path + '/annot_train.csv', header=None, index=None)
test_df.to_csv(keras_path + '/annot_test.csv', header=None, index=None)

/content/drive/My Drive/person_detection/WiderPerson/Annotations
